In [1]:
import pandas as pd
from streamlit_assets.components import get_recos_users
from sklearn.preprocessing import LabelEncoder
from surprise import SVD, NMF, KNNBasic, SVDpp
from surprise.model_selection import cross_validate
from surprise import Dataset
from surprise import Reader
import numpy as np

### Exploring metacritic data for recos

In [2]:
meta_df = pd.read_csv('data/metacritic.csv').dropna(subset='critic_meta_score')
#meta_df = meta_df.rename(columns={"title":"item_id"})

meta_df.describe()


,rank,critic_meta_score,meta_score
count,52689.000000,52689.000000,52689.000000
mean,1554.035472,65.396136,65.277990
std,899.226203,21.106417,14.802795
min,1.000000,0.000000,13.000000
25%,790.000000,50.000000,56.000000
50%,1551.000000,70.000000,66.000000
75%,2318.000000,80.000000,76.000000
max,3139.000000,100.000000,99.000000


In [4]:
id_maker = LabelEncoder()
meta_df['user_id'] = id_maker.fit_transform(meta_df['author'])
meta_df['item_id'] = meta_df['title']
meta_df['rating'] = meta_df['critic_meta_score'].apply(int)

reviews = meta_df[['user_id','item_id','rating']]

In [11]:
tv_list = ['Breaking Bad: Season 5','Rectify: Season 4','Better Call Saul: Season 5','The Leftovers: Season 1']
#new_ids = meta_df.loc[meta_df['title'].isin(tv_list), 'item_id'].unique()

In [12]:
df_recos = get_recos_users(tv_list,reviews, filter_items=True,filter_n=20, default_rating=90)

c:\git\reco_system\streamlit_assets\components.py:82: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_reviews = df.append(new_df).reset_index(drop=True)


In [13]:
df_merged = meta_df.merge(df_recos, on='item_id').drop_duplicates(subset='item_id').sort_values(by='match', ascending=False).reset_index(drop=True)
df_merged.sort_values(by='match', ascending=False)
df_merged[['title','summary','match']]

,title,summary,match
0,The Night Of: Season 1,The limited series created by Steven Zaillian ...,88.524987
1,Master of None: Season 2,Dev (Aziz Ansari) travels to Italy in the seco...,88.022791
2,Atlanta: Season 2,The second season subtitled the Atlanta Robbin...,87.498602
3,Transparent: Season 2,Maura's (Jeffrey Tambor) journey continues in ...,87.378054
4,The Americans: Season 4,Elizabeth and Phillips are given a new dangero...,87.348073
...,...,...,...
1010,Fuller House: Season 1,The Tanner family returns with D.J. Tanner-Ful...,45.874127
1011,October Road: Season 1,A young novelist (Unscripted's Bryan Greenberg...,45.784338
1012,Work It : Season 1,"After they are laid off, Lee Standish (Ben Kol...",42.966910
1013,Dads: Season 1,In this live-action comedy from producer Seth ...,41.023261
